In [25]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random
import os
from datetime import datetime

# Function to convert HSV to RGB
def hsv_to_rgb(h, s, v):
    return tuple(int(i) for i in cv2.cvtColor(np.uint8([[[h, s, v]]]), cv2.COLOR_HSV2BGR)[0][0])

# def apply_watershed(binary, image):
#     # Noise removal with morphological operations
#     kernel = np.ones((1, 1), np.uint8)
#     opening = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel, iterations=2)

#     # Dilate the result to get the sure background region
#     sure_bg = cv2.dilate(opening, kernel, iterations=2)

#     # Distance transform for sure foreground region
#     dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
#     _, sure_fg = cv2.threshold(dist_transform, 0.1 * dist_transform.max(), 255, 0)

#     # Find unknown region by subtracting sure foreground from the background
#     sure_fg = np.uint8(sure_fg)
#     unknown = cv2.subtract(sure_bg, sure_fg)

#     # Marker labeling
#     _, markers = cv2.connectedComponents(sure_fg)

#     # Add 1 to all markers so that the background is not 0 but 1
#     markers = markers + 1

#     # Mark the unknown regions with 0
#     markers[unknown == 255] = 0

#     # Apply the watershed algorithm
#     markers = cv2.watershed(image, markers)

#     # Mark boundaries in the original image
#     image[markers == -1] = [255, 0, 0]  # Boundaries are marked in red

#     return image, markers

def count_ducks(image_path, threshold_range, contour_size_range, watershed_size_range, save=False):
    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Unable to load image from path {image_path}")
        return
    
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Thresholding to create a binary image
    binary = cv2.inRange(gray, threshold_range[0], threshold_range[1])

    # # Apply watershed algorithm to separate touching objects
    # image_watershed, markers = apply_watershed(binary, image.copy())

    # Find contours after watershed
    contours, _ = cv2.findContours(binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    print("Number of contours:", len(contours))

    # Optional: Draw contours with unique colors
    output_image_all_contours = image.copy()
    for cnt in contours:
        hue = random.randint(0, 179)  # OpenCV uses hue values from 0 to 179
        color = hsv_to_rgb(hue, 200, 200)
        cv2.drawContours(output_image_all_contours, [cnt], -1, color, 2)

    # Filter contours based on size and collect areas
    duck_contours = []
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if watershed_size_range[0] < area < watershed_size_range[1]:
            kernel = np.ones((5, 5), np.uint8) 
            cnt_erosion = cv2.erode(cnt, kernel, iterations=1) 
            duck_contours.append(cnt_erosion)

    # Count the number of ducks
    number_of_ducks = len(duck_contours)
    print(f'Number of ducks: {number_of_ducks}')

    # Optional: Draw contours with unique colors
    output_image_filtered_contours = image.copy()
    for cnt in duck_contours:
        hue = random.randint(0, 179)  # OpenCV uses hue values from 0 to 179
        color = hsv_to_rgb(hue, 200, 200)
        cv2.drawContours(output_image_filtered_contours, [cnt], -1, color, 2)

    # Convert BGR images to RGB
    output_image_all_contours = cv2.cvtColor(output_image_all_contours, cv2.COLOR_BGR2RGB)
    output_image_filtered_contours = cv2.cvtColor(output_image_filtered_contours, cv2.COLOR_BGR2RGB)
    gray_rgb = cv2.cvtColor(gray, cv2.COLOR_GRAY2RGB)  # Convert grayscale to RGB for consistent plotting

    # Create a timestamp for saving files
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    if save:
        output_dir = f'outputs/{timestamp}'
        os.makedirs(output_dir, exist_ok=True)

    # Plotting code (same as before)
    figsize = (8,8)

    # # Display grayscale image
    # plt.figure(figsize=figsize)
    # plt.imshow(gray, cmap='gray')
    # plt.axis('off')
    # plt.title('Grayscale Image')
    # if save:
    #     plt.savefig(f'{output_dir}/grayscale.png', bbox_inches='tight', pad_inches=0)
    # plt.show()

    # Display the binary image
    plt.figure(figsize=figsize)
    plt.imshow(binary, cmap='gray')
    plt.title('Binary Image')
    plt.axis('off')
    if save:
        plt.savefig(f'{output_dir}/binary.png', bbox_inches='tight', pad_inches=0)
    plt.show()

    # # Display the image after watershed
    # plt.figure(figsize=figsize)
    # plt.imshow(cv2.cvtColor(image_watershed, cv2.COLOR_BGR2RGB))
    # plt.title(f'Watershed Result')
    # plt.axis('off')
    # if save:
    #     plt.savefig(f'{output_dir}/watershed_result.png', bbox_inches='tight', pad_inches=0)
    # plt.show()

    # Display the image with all contours
    plt.figure(figsize=figsize)
    plt.imshow(output_image_all_contours)
    plt.title(f'All Contours, Number of contours: {len(contours)}')
    plt.axis('off')
    if save:
        plt.savefig(f'{output_dir}/contours_all.png', bbox_inches='tight', pad_inches=0)
    plt.show()

    # Display the image with filtered contours
    plt.figure(figsize=figsize)
    plt.imshow(output_image_filtered_contours)
    plt.title(f'Filtered Contours, Number of ducks: {number_of_ducks}')
    plt.axis('off')
    if save:
        plt.savefig(f'{output_dir}/contours_filtered.png', bbox_inches='tight', pad_inches=0)
    plt.show()

In [26]:
# Example usage
# count_ducks('dataset/overhead_1.png', threshold_range=[25,255], contour_size_range=[10, 100], save=False)
# count_ducks('dataset/overhead_2.png', threshold_range=[50,255], contour_size_range=[0.01, 100], save=False)
count_ducks('dataset/overhead_2_crop.png', threshold_range=[50,255], contour_size_range=[0.01, 100], watershed_size_range=[100, 7000], save=False)
# count_ducks('dataset/cluster_1.jpg', threshold_range=[80,255], contour_size_range=[10, 100], save=False)
# count_ducks('dataset/oblique_1.png', threshold_range=[80,255], contour_size_range=[80, 500], save=False)

Number of contours: 1447


error: OpenCV(4.10.0) D:/a/opencv-python/opencv-python/opencv/modules/imgproc/src/morph.simd.hpp:756: error: (-213:The function/feature is not implemented) Unsupported data type (=12) in function 'cv::opt_AVX2::getMorphologyRowFilter'
